In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Загрузка данных и компонентов
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

# Определение функции токенизации (без padding)
def tokenize_function(examples):
    # padding будет применен позже
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

# Применение токенизации ко всему набору данных
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Переименование колонки, чтобы она соответствовала ожиданиям модели
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Форматирование данных в TensorFlow с использованием collate_fn
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids"],
    label_cols="labels",
    shuffle=True,
    batch_size=8,
    collate_fn=tokenizer.pad,  # Это исправление!
)
tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids"],
    label_cols="labels",
    shuffle=False,
    batch_size=8,
    collate_fn=tokenizer.pad,  # Это исправление!
)


c:\Users\user\.conda\envs\ds-vl2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\.conda\envs\ds-vl2\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [2]:
# Обучение модели по умолчанию
# model.compile(optimizer="adam")
# model.fit(
#     tf_train_dataset,
#     validation_data=tf_validation_dataset,
#     epochs=3,
# )

In [3]:
# задание класса TrainingArguments, который будет содержать все гиперпараметры для Trainer
from transformers import TrainingArguments

# training_args = TrainingArguments("test-trainer")

# Измените путь на что-то вроде C:/Users/user/Desktop/test-trainer
training_args = TrainingArguments("C:/Users/user/Desktop/test-trainer") 

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

c:\Users\user\.conda\envs\ds-vl2\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Предупреждения сообщают, что некоторые веса не будут использованы (как раз тех слоев, которые не будут использоваться) и для новых будут инициализированы случайные веса

После того, как мы загрузили модель, мы можем определить Trainer и передать туда нужные объекты: model, training_args, обучающую и валидационную выборки, data_collator и tokenizer

In [5]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    # data_collator=data_collator,
    tokenizer=tokenizer,
)

c:\Users\user\.conda\envs\ds-vl2\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Для fine-tuning модели на нашем датасете мы просто должны вызвать метод train() у Trainer:

In [6]:
trainer.train()

 36%|███▋      | 500/1377 [00:26<00:44, 19.49it/s]Checkpoint destination directory C:/Users/user/Desktop/test-trainer\checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 0.5436, 'grad_norm': 9.660467147827148, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


 73%|███████▎  | 1000/1377 [00:53<00:19, 19.26it/s]Checkpoint destination directory C:/Users/user/Desktop/test-trainer\checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 0.3086, 'grad_norm': 9.229806900024414, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


100%|██████████| 1377/1377 [01:14<00:00, 18.46it/s]

{'train_runtime': 74.6109, 'train_samples_per_second': 147.485, 'train_steps_per_second': 18.456, 'train_loss': 0.36237642153848665, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.36237642153848665, metrics={'train_runtime': 74.6109, 'train_samples_per_second': 147.485, 'train_steps_per_second': 18.456, 'train_loss': 0.36237642153848665, 'epoch': 3.0})

# Валидация
Давайте посмотрим как мы можем создать и использовать в процессе обучения полезную функцию compute_metrics(). Функция должна принимать на вход объект EvalPrediction (именованный кортеж с полями predictions и label_ids) и возвращать словарь, где ключи - названия метрик, а значения - оценки этих метрик. Чтобы получить предсказания, мы можем использовать функцию Trainer.predict():

In [7]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

100%|██████████| 51/51 [00:00<00:00, 89.67it/s]

(408, 2) (408,)


Как можно заметить, predictions - массив 408 х 2 (408 - число элементов в датасете, который мы использовали). Это логиты для каждого элемента нашего датасета, переданного в predict() (как вы видели в предыдущей главе все модели Трансформеров возвращают логиты). Чтобы превратить их в предсказания и сравнить с нашими лейблами, нам необходимо узнать индекс максимального элемента второй оси:

In [8]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
print(preds)

[1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0
 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 1 0
 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1
 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0
 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0
 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0
 1]


In [9]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8480392156862745, 'f1': 0.8956228956228957}

# Собирая вместе все фрагменты выше, мы получим нашу функцию compute_metrics():

In [13]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
# training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
training_args = TrainingArguments("C:/Users/user/Desktop/test-trainer-2") 
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    # data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
trainer.train()

                                                  
 36%|███▋      | 500/1377 [04:16<00:51, 16.89it/s]

{'loss': 0.5274, 'grad_norm': 3.7457587718963623, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


                                                  
 36%|███▋      | 500/1377 [05:32<00:51, 16.89it/s] 

{'loss': 0.3071, 'grad_norm': 1.0215269327163696, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


                                                  
100%|██████████| 1377/1377 [02:53<00:00,  7.92it/s]

{'train_runtime': 173.7993, 'train_samples_per_second': 63.314, 'train_steps_per_second': 7.923, 'train_loss': 0.34681898905214575, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.34681898905214575, metrics={'train_runtime': 173.7993, 'train_samples_per_second': 63.314, 'train_steps_per_second': 7.923, 'train_loss': 0.34681898905214575, 'epoch': 3.0})